<a href="https://colab.research.google.com/github/pedrosousaalves/monitoramento_eletrico_grupo5/blob/main/Monitoramento_Eletrico_Grupo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# OBS: O código abaixo depende das variáveis globais 'circuitos' e 'limites'
# já definidas na sua estrutura inicial.

def ler_dados_uso(nome_arquivo="uso_diario.txt"):
    """
    Simula a leitura de um arquivo com o tempo de uso diário e a tarifa de energia.
    Retorna um dicionário: {Nome_Circuito: [Tempo_Horas, Tarifa_R$/kWh]}
    """
    # ** SIMULAÇÃO DE ARQUIVO **: Crie um arquivo 'uso_diario.txt' com este conteúdo:
    # Circuito 1; 8.0; 0.75
    # Motor Bomba; 12.0; 0.75

    dados_uso = {}
    try:
        with open(nome_arquivo, "r") as arq:
            for linha in arq:
                # Ignora linhas em branco ou de cabeçalho
                if not linha.strip() or "Circuito" in linha:
                    continue

                partes = [p.strip() for p in linha.split(";")]
                if len(partes) == 3:
                    nome = partes[0]
                    tempo = float(partes[1])
                    tarifa = float(partes[2])
                    dados_uso[nome] = [tempo, tarifa]
    except FileNotFoundError:
        print(f"ERRO: Arquivo '{nome_arquivo}' não encontrado. Crie um arquivo com dados de uso.")
    return dados_uso

def modulo_extra():
    """
    Módulo para calcular o consumo de energia (kWh) e o custo (R$).
    """
    print("\n--- RODANDO MÓDULO DE CONSUMO DE ENERGIA ---")
    dados_uso = ler_dados_uso()

    if not dados_uso:
        return

    # Cabeçalho do relatório
    relatorio = []
    relatorio.append(f"{'Circuito':<20}{'V (V)':>5}{'I (A)':>8}{'FP':>5}{'P (kW)':>8}{'Tempo (h)':>10}{'Consumo (kWh)':>15}{'Custo (R$)':>12}")
    relatorio.append("-" * 90)

    custo_total = 0.0

    for c in circuitos:
        nome, tipo, v, i, fp, f, data = c

        # Verifica se temos dados de uso para este circuito
        if nome in dados_uso:
            tempo_horas, tarifa = dados_uso[nome]

            # 1. Cálculo da Potência Ativa (P em Watts)
            # P = V * I * FP (para circuitos monofásicos ou uma fase de um trifásico)
            # Vamos usar V, I, e FP medidos
            potencia_w = v * i * fp
            potencia_kw = potencia_w / 1000.0

            # 2. Cálculo do Consumo (kWh)
            consumo_kwh = potencia_kw * tempo_horas

            # 3. Cálculo do Custo (R$)
            custo_rs = consumo_kwh * tarifa
            custo_total += custo_rs

            # Adiciona a linha ao relatório
            linha = f"{nome:<20}{v:>5.0f}{i:>8.2f}{fp:>5.2f}{potencia_kw:>8.3f}{tempo_horas:>10.1f}{consumo_kwh:>15.3f}{custo_rs:>12.2f}"
            relatorio.append(linha)

    relatorio.append("-" * 90)
    relatorio.append(f"{'Custo Total (R$):':<75}{custo_total:>14.2f}")

    print("\n".join(relatorio))
    print("\nRelatório de consumo diário concluído.")

# --- INSTRUÇÃO DE TESTE ---
# Para testar, adicione estas linhas ao final do seu notebook (após definir 'circuitos'):
# Crie o arquivo 'uso_diario.txt' (ou simule o conteúdo na função ler_dados_uso)
# modulo_extra()